In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb, parambokeh
import pandas as pd
import dask.dataframe as dd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, PlotSize

hv.extension('bokeh')

ddf = dd.read_parquet('./data/nyc_taxi.parq/').persist()

# Dashboards Visualizing Hundreds of Millions of Datapoints in 30 Lines of Python

### Philipp Rudiger, James A. Bednar and Jean-Luc Stevens

* Jupyter Dashboards is the only way to generate a graphical dashboard layout by interactively positioning them on a grid
* Paramnb is the only param UI extension that has been released and has support for most parameter types but can only be deployed using the obsolete Jupyter Dashboards
* ParamBokeh is pre-release and allows usage both in the notebook and deployment on bokeh server. It can already be used in the notebook and on bokeh server, however in the notebook it cannot easily be used to completely rerender a plot.

## Declaring parameters

* Param is a library providing Parameters, Python attributes with:

  - type and range checking
  - dynamically generated values
  - documentation strings
  - default values
  
* All features are inherited from parent classes if not specified in a subclass.

## An example

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.Range(default=(1, 9), bounds=(1, 9), doc="Filters taxi trips by number of passengers")

### Class level parameters

In [ ]:
NYCTaxiExplorer.alpha

In [ ]:
NYCTaxiExplorer.alpha = 0.5
NYCTaxiExplorer.alpha

### Validation

In [ ]:
NYCTaxiExplorer.alpha = '0'

### Instance parameters

In [ ]:
explorer = NYCTaxiExplorer(alpha=0.5)
explorer.alpha


In [ ]:
NYCTaxiExplorer.alpha

## Declaring plots

* HoloViews and GeoViews provide declarative way to annotate, explore and visualize your data
* Large library of Elements with associated visual representation

In [ ]:
%%opts RGB [width=800 height=475]
points = hv.Points(ddf, kdims=['pickup_x', 'pickup_y'], vdims=['passenger_count'])
selected = points.select(passenger_count=(0, 9))
taxi_trips = datashade(selected, x_sampling=1, y_sampling=1, cmap=cm['fire'])
taxi_trips

In [ ]:
tiles = gv.WMTS(WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/'
                                   'World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'))
tile_options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
tiles = tiles.opts(plot=tile_options)
tiles

## Composition

In [ ]:
tiles * taxi_trips

## An automatic UI

* paramnb and parambokeh provide UIs built on top of param using ipywidgets and bokeh respectively
* Declaration of parameters is independent of the precise UI library used

In [ ]:
paramnb.Widgets(NYCTaxiExplorer)

In [ ]:
parambokeh.Widgets(NYCTaxiExplorer)

## Putting it all together

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.Range(default=(1, 9), bounds=(1, 9))

    def make_view(self, x_range, y_range, alpha, colormap, plot, passengers):
        map_tiles = tiles(style=dict(alpha=alpha), plot=tile_options) 

        points = hv.Points(ddf, kdims=[plot+'_x', plot+'_y'], vdims=['passenger_count'])
        selected = points.select(passenger_count=passengers)
        taxi_trips = datashade(selected, x_sampling=1, y_sampling=1, cmap=colormap,
                               dynamic=False, x_range=x_range, y_range=y_range,
                              width=800, height=475)
        return map_tiles * taxi_trips
    
    def view(self):
        return hv.DynamicMap(self.make_view, streams=[self, RangeXY()])

In [ ]:
explorer = NYCTaxiExplorer()
paramnb.Widgets(explorer, callback=explorer.event)
explorer.view()

In [ ]:
explorer = NYCTaxiExplorer()
parambokeh.Widgets(explorer, callback=explorer.event)
explorer.view()